Chris Holden (ceholden@gmail.com) - https://github.com/ceholden


Chapter 1: Exploring the GDALDataset class
==========================================

## 介绍
GDAL栅格库最基本的组件之一是一个“[类](https://en.wikipedia.org/wiki/Class_(computer_programming))”或一个对象，它存储了一个人可能想要的关于栅格图像的所有信息。这个类，`GDALDataset`，将一个栅格文件的信息与一些人可能想要对一个栅格图像执行的操作结合起来，比如从一个图像中读取。一个类存储的信息通常被称为“[属性](https://en.wikipedia.org/wiki/Property_(programming))”或特征，而一个类可以执行的操作被称为“[方法](https://en.wikipedia.org/wiki/Method_(computer_programming))”。

如果您来自另一种语言，并想了解Python中面向对象编程的概述，请参阅[Python关于类的教程](https://docs.python.org/3/tutorial/classes.html)或[LearnPython类教程](http://www.learnpython.org/en/Classes_and_Objects)。

对于那些希望得到GDAL“应用程序编程接口”([API](http://en.wikipedia.org/wiki/Application_programming_interface))的详细、完整的参考的人，您可以在这里找到C和Python API：

- [C API](http://gdal.org/python/osgeo.gdal.Dataset-class.html)
- [Python API](http://gdal.org/python/osgeo.gdal.Dataset-class.html)


一些类方法包括：

- `GetDriver`
- `GetRasterBand`
- `GetGeoTransform`
- `GetProjection`
- `GetSubDatasets`

这些类方法是所谓的“获取器”方法，它们允许您访问类属性（记住：类属性只是属于类的变量）。当您调用类方法`GetDriver`时，GDAL数据集将返回负责处理该栅格文件格式的输入和输出操作的图像格式驱动程序（例如，ENVI驱动程序，GeoTIFF驱动程序，HDF驱动程序）。类似地，`GetGeoTransform`方法将返回可以用于在像素坐标和投影坐标之间进行转换的变换。


(1) Google Translate. https://translate.google.com/.
(2) DeepL Translate: The world's most accurate translator. https://www.deepl.com/en/translator/l/en/zh.
(3) PONS translations | Best English-Chinese translations online. https://en.pons.com/translate/english-chinese.
(4) Translate English to Chinese (Simplified): Free Online Translation. https://www.translate.com/english-chinese_simplified.

## Python中的模块导入

现在我们已经看到了一些GDALDataset对象如何封装了许多与光栅图像概念相关的思想，让我们看看我们如何在Python中实现这些思想。

在我们开始之前，我们需要告诉Python我们将使用GDAL Python包中的函数、类和变量。这个的技术说法是我们需要将GDAL模块导入到我们的[命名空间](http://en.wikipedia.org/wiki/Namespace)中（参见Python关于`module`系统的文档[这里](https://docs.python.org/2/tutorial/modules.html)）。

我们将使用一些`import`语句来做到这一点：

In [1]:
# Import the Python 3 print function
from __future__ import print_function

# Import the "gdal" submodule from within the "osgeo" module
from osgeo import gdal

# We can check which version we're running by printing the "__version__" variable
print("GDAL's version is: " + gdal.__version__)
print(gdal)

GDAL's version is: 3.4.3
<module 'osgeo.gdal' from 'D:\\Geo_Python_2024\\envi\\lib\\site-packages\\osgeo\\gdal.py'>


Once we import the `gdal` submodule, Python will know where to look on our system for the code that implements the GDAL API. When we want to access classes, variables, or functions within the `gdal` submodule, we will need to reference the full path, which includes the `gdal` reference:

In [2]:
# Let's print out the value of the GDAL Byte data type (GDT_Byte)
#     the number printed out is how GDAL keeps track of the various data types
#     this variable, which has a fixed numeric value, is what's called an enumerated type, or enum

# Works
print(gdal.GDT_Byte)
# Doesn't work
print(GDT_Byte)

1


NameError: name 'GDT_Byte' is not defined

数据类型`GDT_Byte`不存在于*全局*命名空间中。我们需要告诉Python在哪里查找它。

在简要介绍了Python的命名空间设置以及它如何应用于GDAL之后，让我们看一些例子：

### 例子
#### 打开一幅图像
当我们在GDAL中打开一幅图像时，我们创建了一个GDALDataset对象。顾名思义，我们可以使用`gdal`中的"Open"函数来打开一幅图像。

我们将使用本章提供的一个示例图像。这个图像是一个Landsat 7图像的子集，包含了这个传感器上的7个波段，按照波长的顺序重新排列（例如，Landsat 7的第二个SWIR通道在我们的堆栈中位于热通道之前）。这个图像的最后一个波段是一个来自Fmask的云和云影掩码。


In [5]:
# Open a GDAL dataset
dataset = gdal.Open('../../example/LE70220491999322EDC01_stack.gtif', gdal.GA_ReadOnly)

print(dataset)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000024D5EDE8D20> >


Now that we have this dataset open, let's explore some of its capabilities.

#### Image attributes

In [7]:
# 图像属性

# 图像有多少个波段？
num_bands = dataset.RasterCount
print('图像波段数：{n}\n'.format(n=num_bands))

# 有多少行和列？
rows = dataset.RasterYSize
cols = dataset.RasterXSize
print('图像尺寸为：{r} 行 x {c} 列\n'.format(r=rows, c=cols))

# 栅格数据集是否有描述或元数据？
desc = dataset.GetDescription()
metadata = dataset.GetMetadata()

print('栅格数据集描述: {desc}'.format(desc=desc))
print('栅格数据集元数据:')
print(metadata)
print('\n')

# 使用了哪个驱动程序打开了栅格数据集？
driver = dataset.GetDriver()
print('栅格数据集驱动程序: {d}\n'.format(d=driver.ShortName))

# 栅格数据集的投影是什么？
proj = dataset.GetProjection()
print('图像投影:')
print(proj + '\n')

# 栅格数据集的“地理转换”是什么？
gt = dataset.GetGeoTransform()
print('图像地理转换: {gt}\n'.format(gt=gt))


图像波段数：8

图像尺寸为：250 行 x 250 列

栅格数据集描述: ../../example/LE70220491999322EDC01_stack.gtif
栅格数据集元数据:
{'AREA_OR_POINT': 'Area', 'Band_1': 'band 1 reflectance', 'Band_2': 'band 2 reflectance', 'Band_3': 'band 3 reflectance', 'Band_4': 'band 4 reflectance', 'Band_5': 'band 5 reflectance', 'Band_6': 'band 7 reflectance', 'Band_7': 'band 6 temperature', 'Band_8': 'Band 8'}


栅格数据集驱动程序: GTiff

图像投影:
PROJCS["WGS 84 / UTM zone 15N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-93],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32615"]]

图像地理转换: 

这段代码会打印出栅格数据集的地理转换信息，该信息描述了栅格数据的空间参考信息以及像素在地理坐标系统中的位置和分辨率。通常，地理转换信息由6个值组成：

1. 左上角x坐标
2. 水平像素分辨率
3. x轴的旋转（通常为0，表示正北方向）
4. 左上角y坐标
5. y轴的旋转（通常为0，表示正北方向）
6. 垂直像素分辨率（通常为负值，因为像素通常是从上到下编号）

这些值一起描述了栅格数据在地理空间中的位置和取样方式。例如，左上角的x和y坐标给出了栅格数据的起始位置，分辨率定义了每个像素在地理坐标中代表的距离，而旋转参数则描述了数据集的旋转或倾斜。


我们获得的前几个信息都很简单明了——栅格的大小、波段数、描述、元数据和栅格的文件格式。

图像的投影是以一种被称为"通用文本"的格式表示的。关于特定投影的更多信息，以及投影描述格式之间的格式转换（例如，proj4字符串、WKT、ESRI WKT、JSON等），请参见[空间参考]。


我们访问的最后一个信息是一种叫做"地理变换"的东西。这组6个数字提供了从像素坐标到投影坐标的转换所需的所有信息。在这个例子中，第一个数字（462405）和第四个数字（1741815）是栅格左上角像素的左上角。栅格在x和y方向上的像素大小分别列为第二个（30）和第六个（-30）数字。由于我们的栅格是正北方向的，所以第三个和第五个数字是0。关于GDAL数据模型的更多信息，请[访问这个网页]


#### 图像栅格波段

我们创建的 GDALDataset 对象包含了许多有用的信息，但它并不直接用于读取栅格图像。相反，我们需要使用 `GetRasterBand` 方法逐个访问栅格的波段：

(1) GDALDataset C++ API — GDAL documentation. https://gdal.org/api/gdaldataset_cpp.html.
(2) GDAL Python Tutorial: Reading and Writing Raster Datasets. https://opensourceoptions.com/gdal-python-tutorial-reading-and-writing-raster-datasets/.
(3) org.gdal.gdal.Dataset.GetRasterBand java code examples | Tabnine. https://www.tabnine.com/code/java/methods/org.gdal.gdal.Dataset/GetRasterBand.
(4) Fastest way to extract all bands from raster at once (python/gdal). https://stackoverflow.com/questions/47461386/fastest-way-to-extract-all-bands-from-raster-at-once-python-gdal.
(5) Raster API tutorial — GDAL documentation. https://gdal.org/tutorials/raster_api_tut.html.

In [9]:
# Open the blue band in our image
blue = dataset.GetRasterBand(1)

print(blue)

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x0000024D60233BA0> >


根据我们的 GDALDataset 指南，让我们探讨一下 GDALRasterBand 的一些属性和方法：

1. **GetBand(int band_num)**: 通过波段索引获取特定波段。例如，`GetBand(1)` 将返回第一个波段。

2. **ReadAsArray(xoff, yoff, xsize, ysize)**: 从栅格波段中读取数据作为 NumPy 数组。参数 `xoff` 和 `yoff` 是左上角像素的偏移量，`xsize` 和 `ysize` 是要读取的像素数。

3. **GetStatistics(approx_ok=True, force=False)**: 获取波段的统计信息，如最小值、最大值、平均值和标准差。设置 `approx_ok` 为 `True` 可以加快计算，但可能不够精确。

4. **ComputeRasterMinMax(force=False)**: 计算波段的最小值和最大值。如果 `force` 设置为 `True`，则强制重新计算。

5. **GetHistogram(min_val, max_val, num_buckets, include_out_of_range=False, approx_ok=True)**: 获取波段的直方图。参数 `min_val` 和 `max_val` 是直方图的值范围，`num_buckets` 是直方图的桶数。

6. **GetColorTable()**: 获取颜色表（如果存在）。颜色表用于将像素值映射到颜色。

7. **SetNoDataValue(nodata_value)**: 设置波段的无数据值。无数据值通常用于表示缺失或无效的像素。

这些方法和属性将帮助您在处理栅格数据时更好地理解和操作波段。如果您有其他问题或需要更多详细信息，请随时告知！

In [10]:
# 获取波段的数据类型
datatype = blue.DataType
print('波段数据类型: {dt}'.format(dt=blue.DataType))

# 如果您还记得我们讨论过的枚举类型，我们打印的这个“3”对我们来说有一个更有用的定义
datatype_name = gdal.GetDataTypeName(blue.DataType)
print('波段数据类型: {dt}'.format(dt=datatype_name))

# 我们还可以询问这种数据类型占用了多少空间
bytes = gdal.GetDataTypeSize(blue.DataType)
print('波段数据类型大小: {b} 字节\n'.format(b=bytes))

# 那么波段的统计信息呢？
band_max, band_min, band_mean, band_stddev = blue.GetStatistics(0, 1)
print('波段范围: {minimum} - {maximum}'.format(maximum=band_max,
                                             minimum=band_min))
print('波段均值、标准差: {m}, {s}\n'.format(m=band_mean, s=band_stddev))


波段数据类型: 3
波段数据类型: Int16
波段数据类型大小: 16 字节

波段范围: 1810.0 - 198.0
波段均值、标准差: 439.015984, 139.7168287663



请注意，我们不需要将图像读入 Python 的内存中来计算这些统计数据 - GDAL 已经为我们完成了所有这些工作。

然而，对于大多数应用程序，我们需要使用 GDAL 将栅格波段读入内存。当我们将栅格波段加载到内存中时，我们将其读入一个 [NumPy](http://www.numpy.org/) 的二维数组。NumPy 是“用于 Python 科学计算的基本包”，因为它允许我们以非常高效的方式表示数据。

NumPy 数组是科学 Python 套件的基石或构建块。熟悉一下：

+ [NumPy 适用于 MATLAB 用户](http://wiki.scipy.org/NumPy_for_Matlab_Users)
+ [NumPy 教程](http://wiki.scipy.org/Tentative_NumPy_Tutorial)
+ [NumPy API 参考手册](http://docs.scipy.org/doc/numpy/reference/)

就像我们使用 `import` 将 GDAL 中的例程和数据类型提供给我们一样，我们将加载 NumPy。当我们导入 NumPy 时，我们还会给它一个别名，以便我们不必每次使用它时都输入 `numpy`。

(1) NumPy reference — NumPy v1.26 Manual. https://numpy.org/doc/stable/reference/.
(2) NumPy documentation — NumPy v1.26 Manual. https://numpy.org/doc/stable/.
(3) NumPy Reference — NumPy v1.10 Manual - SciPy.org. https://docs.scipy.org/doc/numpy-1.10.1/reference/index.html.
(4) NumPy Reference — NumPy v1.15 Manual - SciPy.org. https://docs.scipy.org/doc/numpy-1.15.1/reference/.
(5) NumPy - Learn. https://numpy.org/learn/.
(6) NumPy Tutorial. https://www.tutorialspoint.com/numpy/index.htm.
(7) NumPy Tutorial - W3Schools. https://www.w3schools.com/python/numpy/default.asp.
(8) NumPy: the absolute basics for beginners — NumPy v1.26 Manual. https://numpy.org/doc/stable/user/absolute_beginners.html.

In [14]:
# No alias
import numpy
print(numpy.__version__)

# Alias or rename to "np" -- a very common practice
import numpy as np
print(np.__version__)

1.24.3
1.24.3


In order to read our band into one of these wonderful `np.array` objects, we will use the `ReadAsArray` method from our `GDALRasterBand` object为了将我们的波段读入这些出色的 `np.array` 对象之一，我们将使用 `GDALRasterBand` 对象的 `ReadAsArray` 方法：
```python
# 获取波段数据
blue_data = blue.ReadAsArray()

# 打印波段数据的形状
print(f"波段数据形状：{blue_data.shape}")

# 打印波段数据的前几行
print("波段数据的前几行：")
print(blue_data[:5, :])
```
这将读取蓝色波段的数据并存储在名为 `blue_data` 的 NumPy 数组中。您可以根据需要进一步处理和分析这些数据。

(1) GDALRasterBand C++ API — GDAL documentation. https://gdal.org/api/gdalrasterband_cpp.html.
(2) python gdal ReadAsArray 按块读取栅格 - CSDN博客. https://blog.csdn.net/dou3516/article/details/104986509.
(3) Reading data from raster band with Java and GDAL?. https://gis.stackexchange.com/questions/35583/reading-data-from-raster-band-with-java-and-gdal.
(4) Raster API tutorial — GDAL documentation. https://gdal.org/tutorials/raster_api_tut.html.
(5) raster - Optimizing Python GDAL ReadAsArray - Geographic Information .... https://gis.stackexchange.com/questions/172666/optimizing-python-gdal-readasarray.

In [15]:
help(blue.ReadAsArray)

Help on method ReadAsArray in module osgeo.gdal:

ReadAsArray(xoff=0, yoff=0, win_xsize=None, win_ysize=None, buf_xsize=None, buf_ysize=None, buf_type=None, buf_obj=None, resample_alg=0, callback=None, callback_data=None) method of osgeo.gdal.Band instance
    Reading a chunk of a GDAL band into a numpy array. The optional (buf_xsize,buf_ysize,buf_type)
    parameters should generally not be specified if buf_obj is specified. The array is returned



`ReadAsArray` 方法接受参数，允许我们使用 X 和 Y 偏移量以及大小来指定栅格波段图像的子集。当您要处理大型图像或使用有限的内存时，请记住这种能力。在这些情况下，如果一次读取整个数据集，您将耗尽内存。相反，一次读取一定数量的列和行的块，执行计算并存储输出，然后处理剩余的图像。

现在，我们将只读取整个图像：
```python
# 获取波段数据
blue_data = blue.ReadAsArray()

# 打印波段数据的形状
print(f"波段数据形状：{blue_data.shape}")

# 打印波段数据的前几行
print("波段数据的前几行：")
print(blue_data[:5, :])
```
这将读取蓝色波段的数据并存储在名为 `blue_data` 的 NumPy 数组中。您可以根据需要进一步处理和分析这些数据。

(1) GDALRasterBand C++ API — GDAL documentation. https://gdal.org/api/gdalrasterband_cpp.html.
(2) python gdal ReadAsArray 按块读取栅格 - CSDN博客. https://blog.csdn.net/dou3516/article/details/104986509.
(3) Reading data from raster band with Java and GDAL?. https://gis.stackexchange.com/questions/35583/reading-data-from-raster-band-with-java-and-gdal.
(4) Raster API tutorial — GDAL documentation. https://gdal.org/tutorials/raster_api_tut.html.
(5) raster - Optimizing Python GDAL ReadAsArray - Geographic Information .... https://gis.stackexchange.com/questions/172666/optimizing-python-gdal-readasarray.
(6) NumPy reference — NumPy v1.26 Manual. https://numpy.org/doc/stable/reference/.
(7) NumPy documentation — NumPy v1.26 Manual. https://numpy.org/doc/stable/.
(8) NumPy Reference — NumPy v1.10 Manual - SciPy.org. https://docs.scipy.org/doc/numpy-1.10.1/reference/index.html.
(9) NumPy Reference — NumPy v1.15 Manual - SciPy.org. https://docs.scipy.org/doc/numpy-1.15.1/reference/.
(10) NumPy - Learn. https://numpy.org/learn/.
(11) NumPy Tutorial. https://www.tutorialspoint.com/numpy/index.htm.
(12) NumPy Tutorial - W3Schools. https://www.w3schools.com/python/numpy/default.asp.
(13) NumPy: the absolute basics for beginners — NumPy v1.26 Manual. https://numpy.org/doc/stable/user/absolute_beginners.html.

In [16]:
blue_data = blue.ReadAsArray()

print(blue_data)
print()
print('Blue band mean is: {m}'.format(m=blue_data.mean()))
print('Size is: {sz}'.format(sz=blue_data.shape))

[[569 526 569 ... 311 289 311]
 [568 589 568 ... 267 332 332]
 [546 525 589 ... 311 311 311]
 ...
 [499 543 478 ... 306 349 372]
 [520 520 543 ... 328 372 393]
 [543 564 543 ... 393 414 436]]

Blue band mean is: 439.015984
Size is: (250, 250)


将数据读入 NumPy 数组后，我们可以将其打印到控制台，甚至对其进行统计。除了帮助我们高效存储大量数据外，NumPy 还可以帮助我们进行一些基本的线性代数、数值运算和汇总统计。

假设我们想将所有波段读入一个三维（nrow x ncol x nband）数据集。我们将从初始化一个三维数组开始。接下来，我们将循环遍历栅格图像数据集中的所有波段，并将它们读入我们新分配的三维数组中：
```python
import numpy as np

# 假设我们有三个波段，每个波段的数据形状为 (nrow, ncol)
nrow, ncol = 100, 100
nband = 3

# 初始化一个三维数组，形状为 (nrow, ncol, nband)
all_bands_data = np.zeros((nrow, ncol, nband))

# 假设我们有一个 GDALDataset 对象，其中包含了三个波段的数据
# 这里用 blue_data、green_data、red_data 代替实际的波段数据
blue_data = np.random.rand(nrow, ncol)
green_data = np.random.rand(nrow, ncol)
red_data = np.random.rand(nrow, ncol)

# 将波段数据读入三维数组
all_bands_data[:, :, 0] = blue_data
all_bands_data[:, :, 1] = green_data
all_bands_data[:, :, 2] = red_data

# 打印三维数组的形状
print(f"三维数组形状：{all_bands_data.shape}")

# 打印三维数组的前几行
print("三维数组的前几行：")
print(all_bands_data[:5, :, :])
```
这将创建一个三维数组 `all_bands_data`，其中包含了三个波段的数据。您可以根据需要进一步处理和分析这些数据。

(1) NumPy - Learn. https://numpy.org/learn/.
(2) NumPy Tutorial. https://www.tutorialspoint.com/numpy/index.htm.
(3) NumPy Tutorial - W3Schools. https://www.w3schools.com/python/numpy/default.asp.
(4) NumPy: the absolute basics for beginners — NumPy v1.26 Manual. https://numpy.org/doc/stable/user/absolute_beginners.html.
(5) NumPy reference — NumPy v1.26 Manual. https://numpy.org/doc/stable/reference/.
(6) NumPy documentation — NumPy v1.26 Manual. https://numpy.org/doc/stable/.
(7) NumPy Reference — NumPy v1.10 Manual - SciPy.org. https://docs.scipy.org/doc/numpy-1.10.1/reference/index.html.
(8) NumPy Reference — NumPy v1.15 Manual - SciPy.org. https://docs.scipy.org/doc/numpy-1.15.1/reference/.

In [17]:
# Initialize a 3d array -- use the size properties of our image for portability!
image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount))

# Loop over all bands in dataset
for b in range(dataset.RasterCount):
    # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
    band = dataset.GetRasterBand(b + 1)

    # Read in the band's data into the third dimension of our array
    image[:, :, b] = band.ReadAsArray()

print(image)
print(image.dtype)

[[[ 569.  886.  758. ...  988. 2686.    0.]
  [ 526.  886.  758. ...  988. 2686.    0.]
  [ 569.  798.  776. ...  961. 2686.    0.]
  ...
  [ 311.  452.  356. ...  780. 2487.    0.]
  [ 289.  452.  297. ...  647. 2487.    0.]
  [ 311.  430.  337. ...  674. 2436.    0.]]

 [[ 568.  886.  815. ... 1014. 2686.    0.]
  [ 589.  929.  853. ... 1014. 2686.    0.]
  [ 568.  907.  910. ... 1014. 2686.    0.]
  ...
  [ 267.  430.  318. ...  674. 2487.    0.]
  [ 332.  452.  337. ...  594. 2487.    0.]
  [ 332.  452.  395. ...  727. 2487.    0.]]

 [[ 546.  864.  834. ...  988. 2637.    0.]
  [ 525.  886.  834. ...  988. 2637.    0.]
  [ 589.  928.  891. ...  988. 2637.    0.]
  ...
  [ 311.  452.  356. ...  727. 2487.    0.]
  [ 311.  475.  395. ...  727. 2487.    0.]
  [ 311.  475.  356. ...  753. 2487.    0.]]

 ...

 [[ 499.  851.  766. ... 1124. 2785.    0.]
  [ 543.  829.  747. ... 1387. 2735.    0.]
  [ 478.  829.  766. ... 1414. 2735.    0.]
  ...
  [ 306.  473.  336. ...  753. 2537.    

One minor tweak we can make is to ensure that we read our GDAL image into a NumPy array of a matching datatype. GDAL has a function which can make this `GDAL` <-> `NumPy` translation for us:

In [18]:
dataset.GetRasterBand(1).DataType

3

In [19]:
from osgeo import gdal_array

# DataType is a property of the individual raster bands
image_datatype = dataset.GetRasterBand(1).DataType

# Allocate our array, but in a more efficient way
image_correct = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                 dtype=gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype))

# Loop over all bands in dataset
for b in range(dataset.RasterCount):
    # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
    band = dataset.GetRasterBand(b + 1)

    # Read in the band's data into the third dimension of our array
    image_correct[:, :, b] = band.ReadAsArray()

print("Compare datatypes: ")
print("    when unspecified: {dt}".format(dt=image.dtype))
print("    when specified: {dt}".format(dt=image_correct.dtype))

Compare datatypes: 
    when unspecified: float64
    when specified: int16


这样做效率更高 - 我们节省了 4 倍的内存！

在进入下一章之前，最后一个需要理解的关键概念是如何在 Python 中释放内存。

为了关闭您的 GDAL 数据集并表示您的 NumPy 数组可以被释放，您可以将它们设置为 `None`：
```python
# 关闭 GDAL 数据集
dataset = None

# 释放 NumPy 数组
all_bands_data = None
blue_data = None
green_data = None
red_data = None
```

这将确保您的内存得到适当的释放，以便在处理大型数据集时不会出现内存问题。

In [21]:
dataset = None

image = None
image_correct = None

下一章（[网页链接](chapter_2_indices.html)或[Notebook链接](chapter_2_indices.ipynb)）将运用这些知识来计算归一化植被指数（NDVI）。